In [1]:
import os
import json
import pandas as pd
import traceback



In [2]:
from langchain_openai import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY=os.getenv("api_key")

In [5]:
llm=ChatOpenAI(
               model="deepseek/deepseek-r1:free",   
               openai_api_key=KEY ,  
               openai_api_base="https://openrouter.ai/api/v1",
               temperature=0.7,          
                )

In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000025B23CC9940>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000025B23CCA3C0>, root_client=<openai.OpenAI object at 0x0000025B23B32F90>, root_async_client=<openai.AsyncOpenAI object at 0x0000025B23CCA120>, model_name='deepseek/deepseek-r1:free', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain  
import PyPDF3

In [8]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.

### RESPONSE_JSON
{response_json}
 "IMPORTANT: Respond with only valid JSON exactly matching RESPONSE_JSON. Do NOT add any explanation."

"""

In [10]:

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [11]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_11164\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [13]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [14]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)

In [15]:

#from langchain.schema.runnable import RunnableSequence   
#generate_evaluate_chain = quiz_chain | review_chain

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [16]:
file_path =r"C:\Users\Hp\mcqgen\data.txt"

In [17]:
file_path  

'C:\\Users\\Hp\\mcqgen\\data.txt'

In [18]:
with open(file_path, "r", encoding="utf-8") as file:
    TEXT = file.read()

In [19]:
print(TEXT)

Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.[7][14][15]

Generative AI has

In [20]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5
SUBJECT="generativeai"
TONE="simple"

In [22]:
inputs = {
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON),
}
response = generate_evaluate_chain.invoke(inputs)  
response 



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI i

{'text': 'Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]\n\nGenerative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.[7][14][15]\n\nGen

In [23]:
import json

raw_quiz = response.get("quiz", "").strip()

# Try parsing JSON safely
try:
    if raw_quiz.startswith("```"):
        raw_quiz = raw_quiz.strip("`")
        if raw_quiz.startswith("json"):
            raw_quiz = raw_quiz[4:].strip()
    
    quiz = json.loads(raw_quiz)
except json.JSONDecodeError as e:
    print("Failed to decode quiz JSON:", e)
    print("Raw output was:", raw_quiz)
    quiz = {}


In [24]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [25]:
quiz_table_data

[{'MCQ': 'What is the main purpose of generative artificial intelligence?',
  'Choices': 'a: To analyze data patterns for predictions | b: To generate new data like text, images, or videos | c: To replace all human jobs automatically | d: To develop hardware for computers',
  'Correct': 'b'},
 {'MCQ': 'Which technology enabled the AI boom in the 2020s that improved generative AI tools?',
  'Choices': 'a: Markov chains | b: Symbolic AI methods | c: Transformer-based deep neural networks | d: Quantum computing',
  'Correct': 'c'},
 {'MCQ': 'What ethical concern is NOT associated with generative AI?',
  'Choices': 'a: Creation of deepfakes | b: Violation of intellectual property laws | c: Mass replacement of human jobs | d: Improved healthcare diagnostics',
  'Correct': 'd'},
 {'MCQ': 'What was the first example of algorithmically generated media mentioned in the text?',
  'Choices': "a: AARON's paintings | b: ChatGPT | c: Markov chain-based text | d: Eugeny Onegin novel",
  'Correct': 'c

In [26]:
quiz=pd.DataFrame(quiz_table_data)


In [27]:
quiz.to_csv("Generativeai.csv",index=False)